# Ranking evaluation metrics

In this lab you will implement ranking evaluation metrics.

For each query the search engine returns the sorted list of documents. We expect to have relevant documents at the top. 
In supervision learning to evaluate the quality we need labeled data.

Read this article first:
http://queirozf.com/entries/evaluation-metrics-for-ranking-problems-introduction-and-examples
https://medium.com/swlh/rank-aware-recsys-evaluation-metrics-5191bba16832

## Binary relevance metrics

We will start from the assumption, that each document is either relevant or not.

#### recall, precision, f1

First metrics are already faminiar for you from Introduction to Machine Learning course.
Implement relall, precision, and relevance for top $k$ documents.

relevance is a list which represents that document with this index in ranking is relevant or not.

In [18]:
import numpy as np
import math

def recall_at_k(relevance, k):
    return sum(relevance[:k]) / (sum(relevance[:k]) + sum(relevance[k:]))

def precision_at_k(relevance, k):
    return sum(relevance[:k])/k

def f1_at_k(relevance, k):
    r, p = precision_at_k(relevance, k), recall_at_k(relevance, k)
    return 2 * r * p / (r + p)

In [19]:
r = [1, 0, 1, 1, 0, 1, 0, 0]

print(recall_at_k(r, 1))
print(recall_at_k(r, 8))

print(precision_at_k(r, 1))
print(precision_at_k(r, 7))

print(f1_at_k(r, 1))
print(f1_at_k(r, 8))

0.25
1.0
1.0
0.5714285714285714
0.4
0.6666666666666666


#### Average Precision
You can calculate the AP using the following algorithm:

<img src="http://queirozf.com/images/contents/mnc7sx1.png">

In [47]:
def average_precision(relevance, k):
    running_sum = np.array(np.cumsum(relevance[:k])) * np.array(relevance[:k])
    running_sum = running_sum / np.array([i for i in range(1,k+1,1)])
    
    return sum(running_sum) / sum(relevance[:k])

In [48]:
print(*["Rank " + str(i)+ ": " + str(average_precision(r, i)) for i in range(1,9)], sep='\n')


Rank 1: 1.0
Rank 2: 1.0
Rank 3: 0.8333333333333333
Rank 4: 0.8055555555555555
Rank 5: 0.8055555555555555
Rank 6: 0.7708333333333333
Rank 7: 0.7708333333333333
Rank 8: 0.7708333333333333


## Relevance as a real number

#### DCG

DCG - discounted cumulative gain, does't require the relevance to be a binary feature. In many situations one document is more relevant than another and we want to represent it in supervised evaluation. Often the relevance is a number form ${0,1,2,3}$, but ${0,1}$ is also appropriate for usage.

The idea is that each rerelvant document brings a "gain" for a user. He or she looks through the documents from the first. So the gain sums cumulatively. But it is better to have a relevant document at the top of the ranking, so the weight of that gain decreases, or we have a discounded weight with increasing of the document position. And since the weight is decreasing, we can calculate this value for the top $k$ documents in ranking.

$$DCG@k = \sum_{1}^{k}\frac{2^{rel_i}-1}{log_2(i+1)}$$

In [59]:
def dcg_at_k(relevance, k=10):
    relevance = np.array(relevance)
    #relevance = relevance / np.sqrt(np.dot(relevance, relevance))
    return sum(map(lambda rel, i: (2 ** rel - 1) / math.log2(i+2), 
                   relevance[:k], [i for i in range(k)]))
    

In [60]:
r2 = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]


print(dcg_at_k(r, 1))
print(dcg_at_k(r, 3))
print(dcg_at_k(r, 8))

print(dcg_at_k(r2, 1))
print(dcg_at_k(r2, 10))

1.0
1.5
2.2868837451814152
7.0
16.80260104782745


#### nDCG
Now the idea is to normalize it to the maximum value.

In [73]:
def ndcg_at_k(relevance, k=10):
    sorted_relevance = list(reversed(sorted(relevance)))
    return dcg_at_k(relevance, k) / dcg_at_k(sorted_relevance,k)

In [75]:
print(*["Rank " + str(i)+ ": " + str(ndcg_at_k(r, i)) for i in range(1,len(r) + 1)], sep='\n')

Rank 1: 1.0
Rank 2: 0.6131471927654584
Rank 3: 0.7039180890341347
Rank 4: 0.75369761125927
Rank 5: 0.75369761125927
Rank 6: 0.8927537907700456
Rank 7: 0.8927537907700456
Rank 8: 0.8927537907700456


## Test in real data

You already have search engines on songs or news. Test the evaluation on real data.

1. Choose the search query query
2. Run the search
3. Manually look top 10 results and evaluate each of them if it relevant or not
4. Calculate AP and DCG (relevance is either 0 or 1)

In [76]:
query = "food aid"